# AI for Autonomous Vehicles
Brought to you by Daniel Sikar - daniel.sikar@city.ac.uk
and
City University of London Data Science Society - https://www.datasciencesociety.city/

## Predicting steering angles from images using Convolutional Neural Networks

Notebook: https://github.com/dsikar/ai-for-autonomous-vehicles/AIForAutonomousVehicles.ipynb

## Learning objectives
1. Gain familiarity with Google Colab and Jupyter Notebooks
2. Load a pre-train model, make predictions and assess results
3. Train a model to make predictions and assess results

## Instructions
Run each "code" cell sequencially from top to bottom, by clicking on each cell then click "play" button or keying Ctrl/Control + RETURN keys.

# 1. Getting to know the environment
A number of commented commands that can be run in a code cell.

In [ ]:
# How much space have we got?
# !df -h
# Command help
# !man df
# What is on the filesystem?
!ls
# Where are we?
# !pwd
# What does filesystem look like - / ?
# !ls /
# Downloading notebook:
# File > "Download .ipynb"
# or data on filesystem:
# from google.colab import files
# files.download('nlp-model.h5') 

# 2. Get the data
Download the the shared Google drive file.

In [ ]:
# Install PyDrive
!pip install PyDrive

# Import modules
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Get the shareable link e.g. https://drive.google.com/file/d/1hDvGznkneVMVNNHAV98gIUuEW_tXC7z4/view?usp=sharing
# Get the id from the link 1hDvGznkneVMVNNHAV98gIUuEW_tXC7z4
downloaded = drive.CreateFile({'id':"1hDvGznkneVMVNNHAV98gIUuEW_tXC7z4"})   
# For consistency, we use the same name as file uploaded to google drive
downloaded.GetContentFile('ai-av-dataset.tar.gz')

# 3. Examine and decompress downloaded data
Our dataset will be in "dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020" directory, while our pre-trained models in "models" directory.  
The dataset was generated by [SDSandbox](https://github.com/tawnkramer/sdsandbox/), a [Unity](https://unity.com/) (game engine) based self-driving training and testing environment.

In [ ]:
# !ls
!tar xvf ai-av-dataset.tar.gz
# !ls
# !ls models
# !ls dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/*.jpg
# !ls dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/*.jpg | wc -l

# 4. Examine one steering angle "label"
Each image e.g. "100_cam-image_array_.jpg" will have a corresponding label e.g. "record_100.json"

In [ ]:
# !ls dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/100_cam-image_array_.jpg
# !ls dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/record_100.json
!cat dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/record_100.json

# 5. Plot steering angles from testing dataset
Go through all .json files, extracting the "user/angle" value, multiplying by 25 and plotting. Note, 25 is the maximum steering angle in this case.

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import statistics
import seaborn as sns
import os
import fnmatch

def GetJSONSteeringAngles(filemask):
    """
    Get steering angles stored as 'user/angle' attributes in .json files
    Inputs:
        filemask: string, path and mask
    Outputs
        svals: list, steering values
    """
    filemask = os.path.expanduser(filemask)
    path, mask = os.path.split(filemask)

    matches = []
    for root, dirnames, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, mask):
            matches.append(os.path.join(root, filename))
    # sort by create date
    # matches = sorted(matches, key=os.path.getmtime)
    # if create date lost, sort by string converted to number
    matches = str2numsort(matches)
    # steering values
    svals = []
    for fullpath in matches:
            frame_number = os.path.basename(fullpath).split("_")[0]
            json_filename = os.path.join(os.path.dirname(fullpath), "record_" + frame_number + ".json")
            jobj = load_json(json_filename)
            svals.append(jobj['user/angle'])
    return svals

def GetJPGFiles(filemask):
    """
    Get list of .jpg files
    Inputs:
        filemask: string, path and mask
    Outputs
        matches: list, file paths
    """
    filemask = os.path.expanduser(filemask)
    path, mask = os.path.split(filemask)

    matches = []
    for root, dirnames, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, mask):
            matches.append(os.path.join(root, filename))
    # sort by create date
    # matches = sorted(matches, key=os.path.getmtime)
    # if create date lost, sort by string converted to number
    matches = str2numsort(matches)
    return matches
    
def str2numsort(matches):
    """
    Sort a known list of strings by number, this only works for 223_cam-image_array_.jpg.
    Adjust the index in second .split() to adapt
    Inputs
        matches: list of strings, path to file
    Outputs
        matches: list of sorted strings
    Example
    matches = str2numsort(matches)
    """
    # init empty list
    mymatches = []
    for i in range (0,len(matches)):
        # extra the integer part of file e.g. 223 from '223_cam-image_array_.jpg' 
        # and append together with filename matches[i] to new list mymatches
        mymatches.append([matches[i], int(matches[i].split('/')[-1].split('_')[0])])
    # sort by second list array element
    mymatches = sorted(mymatches, key=lambda x: x[1]);
    # clear unsorted list
    matches = []
    # repopulated with sorted strings only
    for i in range (0,len(mymatches)):
        # append filename only
        matches.append(mymatches[i][0])
    return matches

def load_json(filepath):
    """
    Load a json file
    Inputs
        filepath: string, path to file
    Outputs
        data: dictionary, json key, value pairs
    Example
    path = "~/git/msc-data/unity/roboRacingLeague/log/logs_Sat_Nov_14_12_36_16_2020/record_11640.json"
    js = load_json(path)
    """
    with open(filepath, "rt") as fp:
        data = json.load(fp)
    return data

# plot ground truth steering angles for
filemask = './dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/*.jpg'
g = GetJSONSteeringAngles(filemask)
# print(type(g)) # list
g = np.asarray(g)
# print(type(g)) # <class 'numpy.ndarray'>
plt.rcParams["figure.figsize"] = (18,3)
nc = 25 # norm. constant, maximum steering angle

plt.plot(g*nc)
# plt.plot(sarr[:,1]*25, label="simulator")

plt.ylabel('Steering angle')
plt.xlabel('Frame number')    
# Set a title of the current axes.
mytitle = 'Ground truth steering for logs_Wed_Nov_25_23_39_22_2020 - One lap recorded from Generated Track in "Auto Drive w Rec" mode'
plt.title(mytitle)
plt.grid(axis='y')
# set limit
plt.xlim([-5,len(g)+5])
plt.gca().invert_yaxis()
plt.show() 

#6. What versions of Python, Tensorflow and Keras are being used by Google Colab?
These are the programming language and libraries used for generating the model.

In [ ]:
import keras
import tensorflow
!python --version
print("TensorFlow version:", tensorflow.__version__)
print("Keras version:", keras.__version__)

# 7. Load a pre-trained model
Here we load a pre-trained model, keeping in mind that input (image) geometries are different for each of the pre-trained models.

In [ ]:
# !ls models
# 20201207192948_nvidia2.h5 - expected image size 200x66 pixels
# 20201207091932_nvidia1.h5 - expected image size 160x120 pixels
from keras.models import load_model
model=load_model('./models/20201207091932_nvidia1.h5')
model.compile("sgd", "mse")	

# 8. Show predicted steering angle, ground truth steering angle label and corresponding image 

In [ ]:
# 4. Display image and predict
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
fullpath = 'dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/100_cam-image_array_.jpg'
img_arr = cv2.imread(fullpath)
img_arr = cv2.resize(img_arr, (160, 120), cv2.INTER_AREA)
plt.imshow(img_arr)
img_arr = img_arr.reshape((1,) + img_arr.shape)
mypred = model.predict(img_arr)
# Output is an array with two elements, steering angle and throttle
print(mypred)
# We multiply by 25 because the steering angles in .json files are divided by 25 (maximum steering angle)
print(mypred[0][0]*25)
# Steering angle
!cat dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/record_100.json

#9. Load image pre-processing (Augmentation) library

In [ ]:
# code from 
# https://github.com/naokishibuya/car-behavioral-cloning
import cv2, os
import numpy as np
import matplotlib.image as mpimg

IMAGE_HEIGHT, IMAGE_WIDTH =  120, 160
IMAGE_HEIGHT_NET, IMAGE_WIDTH_NET =  120, 160
CROP_TOP, CROP_BOTTOM = 60, -25

def load_image(image_path):
    """
    Load RGB images from a file
    """
    return mpimg.imread(image_path)


def crop(image):
    """
    Crop the image (removing the sky at the top and the car front at the bottom)
    """
    return image[CROP_TOP:CROP_BOTTOM, :, :] # remove the sky and the car front


def resize_net(image):
    """
    Resize the image to the input shape used by the network model
    """
    return cv2.resize(image, (IMAGE_WIDTH_NET, IMAGE_HEIGHT_NET), cv2.INTER_AREA)

def resize_orig(image):
    """
    Resize the image to the original game engine output shape
    """
    return cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), cv2.INTER_AREA)    


def rgb2yuv(image):
    """
    Convert the image from RGB to YUV (This is what the NVIDIA model does)
    """
    return cv2.cvtColor(image, cv2.COLOR_RGB2YUV)


def preprocess(image_path):
    """
    Combine all preprocess functions into one
    """
    image = load_image(image_path)
    image = resize_orig(image)
    image = crop(image)
    image = resize_net(image)
    image = rgb2yuv(image)
    return image


# 10. Show predicted steering angle, ground truth steering angle label and corresponding pre-processed image 

In [ ]:
# 5. Display image and predict, with preprocessing
fullpath = 'dataset/testing/genTrackOneLap/logs_Wed_Nov_25_23_39_22_2020/100_cam-image_array_.jpg'
img_arr = cv2.imread(fullpath)
img_arr = cv2.resize(img_arr, (160, 120), cv2.INTER_AREA)
img_arr = preprocess(fullpath)
plt.imshow(img_arr)
img_arr = img_arr.reshape((1,) + img_arr.shape)
mypred = model.predict(img_arr)
print(mypred)
print(mypred[0][0]*25)

#11. Run predictions with no image pre-processing for testing lap (logs_Wed_Nov_25_23_39_22_2020)
Note: the model was trained with pre-processing, pre-processed images are expected to produce more accurate predictions.

In [ ]:
# from keras.models import load_model
# model=load_model('ai-av-model.h5')
# model.compile("sgd", "mse")
filemask = 'dataset/testing/*.jpg'
files = GetJPGFiles(filemask)
# predPreProc = []
predNoPreProc = []
for file in files:
  fullpath = file
  img_arr = cv2.imread(fullpath)
  img_arr = cv2.resize(img_arr, (160, 120), cv2.INTER_AREA)
  img_arr = img_arr.reshape((1,) + img_arr.shape)
  mypred = model.predict(img_arr)
  predNoPreProc.append(mypred[0][0]) # append steering angle only
print('*** Predictions completed ***')  

#12. Plot results
Note plot does not follow ground truth plot, obtained in step 5. Most predictions are outside the maximum steering angle.

In [ ]:
# plot
pnp = np.asarray(predNoPreProc)
# print(type(g)) # <class 'numpy.ndarray'>
plt.rcParams["figure.figsize"] = (18,3)
nc = 25 # norm. constant, maximum steering angle

plt.plot(pnp*nc)
# plt.plot(sarr[:,1]*25, label="simulator")

plt.ylabel('Steering angle')
plt.xlabel('Frame number')    
# Set a title of the current axes.
mytitle = 'Predicted steering angles for logs_Wed_Nov_25_23_39_22_2020'
plt.title(mytitle)
plt.grid(axis='y')
# set limit
plt.xlim([-5,len(pnp)+5])
plt.gca().invert_yaxis()
plt.show()

#13. Run predictions with image pre-processing for testing lap (logs_Wed_Nov_25_23_39_22_2020)

In [ ]:
files = GetJPGFiles(filemask)
predPreProc = []
# predNoPreProc = []
for file in files:
  fullpath = file
  img_arr = preprocess(fullpath)
  img_arr = img_arr.reshape((1,) + img_arr.shape)
  mypred = model.predict(img_arr)
  predPreProc.append(mypred[0][0]) # append steering angle only
print('*** Predictions with pre-processing completed ***')

#14. Plot results
Note this plot does follow the general trend of ground truth plot, but is "centered" around a negative value, all values being outside the Unity maximum steering angle, hence the model is also unusable.

In [ ]:
# plot
ppr = np.asarray(predPreProc)
# print(type(g)) # <class 'numpy.ndarray'>
plt.rcParams["figure.figsize"] = (18,3)
nc = 25 # norm. constant, maximum steering angle

plt.plot(ppr*nc)
# plt.plot(sarr[:,1]*25, label="simulator")

plt.ylabel('Steering angle')
plt.xlabel('Frame number')    
# Set a title of the current axes.
mytitle = 'Ground truth steering for logs_Wed_Nov_25_23_39_22_2020 - One lap recorded from Generated Track in "Auto Drive w Rec" mode'
plt.title(mytitle)
plt.grid(axis='y')
# set limit
plt.xlim([-5,len(ppr)+5])
plt.gca().invert_yaxis()
plt.show()

#15. Goodness-of-steer
How well/badly is my algorithm driving?

$ g_s(p,g) = \frac{\sum_{i=1}^N \lvert p(i)-g(i) \rvert }{N} \times n_c $

where $p,g$ are prediction and ground truth arrays,  $N$ is the number of predictions and $n_c$ is the normalisation constant, which in this case is the maximum steering angle for the SDSandbox simulated vehicle (+- 25 degrees). In plain english, $g_s$ is defined as the sum of the absolute value of the difference between prediction and ground truth values, divided by the number of predictions multiplied by a normalization constant, that is the steering error average over all predictions. 

In [ ]:
def gos(p, g, n):
    """
    Calculate the goodness-of-steer between a prediction and a ground truth array.
    Inputs
        p: array of floats, steering angle prediction
        g: array of floats, steering angle ground truth.
        n: float, normalization constant
    Output
        gos: float, average of absolute difference between ground truth and prediction arrays
    """
    # todo add type assertion
    assert len(p) == len(g), "Arrays must be of equal length"
    return sum(abs(p - g)) / len(p) * n

fpnp = gos(pnp, g, 25)
fppr = gos(ppr, g, 25)
str_pnp = "{:.2f}".format(fpnp)
str_ppr = "{:.2f}".format(fppr)
print("gs, no pre-processing = ", str_pnp)
print("gs, with pre-processing = ", str_ppr)

#16. Overlay plots for further analysis
We overlay the 2 predictions plus ground truth for some visual analysis.

In [ ]:
def plotMultipleSteeringAngles(p1, p2, p3, n, save=False, track= "Track Name", mname="model.h5", w=18, h=3):
    """
    Plot multiple predicted steering angles
    Inputs
        p: list of tuples, steering angles and labels
        n: integer, 
    """
    plt.rcParams["figure.figsize"] = (18,3)

    
    plt.plot(p1*25)
    plt.plot(p2*25)
    plt.plot(p3*25)
    # plt.plot(sarr[:,1]*25, label="simulator")

    plt.ylabel('Steering angle')
    plt.xlabel('Frame number')    
    # Set a title of the current axes.
    mytitle = 'Ground truth and predicted steering angles: ' + str(track) + ' model ' + str(mname)
    plt.title(mytitle)
    # show a legend on the plot
    plt.legend(['No pre-processing (gs = ' + str_pnp + ')', 'With pre-processing (gs = ' + str_ppr + ')', 'Ground Truth'])
    # Display a figure.
    # horizontal grid only
    plt.grid(axis='y')
    # set limit
    plt.xlim([-5,len(p1)+5])
    plt.gca().invert_yaxis()
    plt.show() 

plotMultipleSteeringAngles(pnp, ppr, g, 25, save=False, track= "Generated Track,", mname="20201207091932_nvidia1.h5", w=18, h=3)

Conclusion: something seems to have gone seriously wrong with the model imported into Colab.
This could be due to different Tensorflow/Keras/Python or other libraries skewing predictions.
Let's reset and try to train one model from scratch.
#17. Train a model

In [ ]:
# code from
# https://github.com/tawnkramer/sdsandbox

from __future__ import print_function
import os
import sys
import glob
import time
import fnmatch
import argparse
import random
import json
import matplotlib.pyplot as plt

import numpy as np
from PIL import Image
from tensorflow import keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
from tensorflow.keras.layers import Dense, Lambda, ELU
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import Cropping2D
from tensorflow.keras.optimizers import Adadelta, Adam

def shuffle(samples):
    '''
    randomly mix a list and return a new list
    '''
    ret_arr = []
    len_samples = len(samples)
    while len_samples > 0:
        iSample = random.randrange(0, len_samples)
        ret_arr.append(samples[iSample])
        del samples[iSample]
        len_samples -= 1
    return ret_arr

def load_json(filename):
    with open(filename, "rt") as fp:
        data = json.load(fp)
    return data

def generator(samples, batch_size=64,):
    '''
    Rather than keep all data in memory, we will make a function that keeps
    it's state and returns just the latest batch required via the yield command.
    
    As we load images, we can optionally augment them in some manner that doesn't
    change their underlying meaning or features. This is a combination of
    brightness, contrast, sharpness, and color PIL image filters applied with random
    settings. Optionally a shadow image may be overlayed with some random rotation and
    opacity.
    We flip each image horizontally and supply it as a another sample with the steering
    negated.
    '''
    num_samples = len(samples)
    
    while 1: # Loop forever so the generator never terminates
        samples = shuffle(samples)
        #divide batch_size in half, because we double each output by flipping image.
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            controls = []
            for fullpath in batch_samples:
                try:
                
                    frame_number = os.path.basename(fullpath).split("_")[0]
                    json_filename = os.path.join(os.path.dirname(fullpath), "record_" + frame_number + ".json")
                    data = load_json(json_filename)
                    steering = float(data["user/angle"])
                    throttle = float(data["user/throttle"])
                
                    try:
                        image = Image.open(fullpath)
                    except:
                        print('failed to open', fullpath)
                        continue

                    #PIL Image as a numpy array
                    image = np.array(image, dtype=np.float32)

                    images.append(image)
                    
                    if 2 == 2:
                        controls.append([steering, throttle])
                    elif 2 == 1:
                        controls.append([steering])
                    else:
                        print("expected 1 or 2 outputs")

                except Exception as e:
                    print(e)
                    print("we threw an exception on:", fullpath)
                    yield [], []


            # final np array to submit to training
            X_train = np.array(images)
            y_train = np.array(controls)
            yield X_train, y_train


def get_files(filemask):
    '''
    use a filemask and search a path recursively for matches
    '''
    #matches = glob.glob(os.path.expanduser(filemask))
    #return matches
    filemask = os.path.expanduser(filemask)
    path, mask = os.path.split(filemask)
    
    matches = []
    for root, dirnames, filenames in os.walk(path):
        for filename in fnmatch.filter(filenames, mask):
            matches.append(os.path.join(root, filename))
    return matches


def train_test_split(lines, test_perc):
    '''
    split a list into two parts, percentage of test used to seperate
    '''
    train = []
    test = []

    for line in lines:
        if random.uniform(0.0, 1.0) < test_perc:
            test.append(line)
        else:
            train.append(line)

    return train, test

def make_generators(inputs, limit=None, batch_size=64):
    '''
    load the job spec from the csv and create some generator for training
    '''
    
    #get the image/steering pairs from the csv files
    lines = get_files(inputs)
    print("found %d files" % len(lines))

    if limit is not None:
        lines = lines[:limit]
        print("limiting to %d files" % len(lines))
    
    train_samples, validation_samples = train_test_split(lines, test_perc=0.2)

    print("num train/val", len(train_samples), len(validation_samples))
    
    # compile and train the model using the generator function
    train_generator = generator(train_samples, batch_size=batch_size)
    validation_generator = generator(validation_samples, batch_size=batch_size)
    
    n_train = len(train_samples)
    n_val = len(validation_samples)
    
    return train_generator, validation_generator, n_train, n_val


def go(model_name, epochs=50, inputs='./log/*.jpg'):

    print('working on model', model_name)

    '''
    modify config.json to select the model to train.
    '''
    model = get_nvidia_model(2)

    '''
    display layer summary and weights info
    '''
    #models.show_model_summary(model)

    callbacks = [
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, verbose=0),
        keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss', save_best_only=True, verbose=0),
    ]
    
    batch_size = 128


    #Train on session images
    train_generator, validation_generator, n_train, n_val = make_generators(inputs, limit=None, batch_size=batch_size)

    if n_train == 0:
        print('no training data found')
        return

    steps_per_epoch = n_train // batch_size
    validation_steps = n_val // batch_size

    print("steps_per_epoch", steps_per_epoch, "validation_steps", validation_steps)

    history = model.fit_generator(train_generator, 
        steps_per_epoch = steps_per_epoch,
        validation_data = validation_generator,
        validation_steps = validation_steps,
        epochs=epochs,
        verbose=1,
        callbacks=callbacks)
    
    try:
        if 1 == 1:
            # summarize history for loss
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.plot(history.history['acc'])
            plt.plot(history.history['val_acc'])            
            plt.title('model loss/acc')
            plt.ylabel('loss/acc')
            plt.xlabel('epoch')
            plt.legend(['train loss', 'test loss', 'train acc', 'test acc'], loc='upper left')
            # plt.savefig('loss.png')
    except:
        print("problems with loss graph")

    model.save('ai-av-model.h5')

def get_nvidia_model(num_outputs):
    '''
    this model is inspired by the NVIDIA paper
    https://images.nvidia.com/content/tegra/automotive/images/2016/solutions/pdf/end-to-end-dl-using-px.pdf
    Activation is RELU
    '''
    # row, col, ch = conf.row, conf.col, conf.ch
    row, col, ch = 120, 160, 3
    
    drop = 0.1
    
    img_in = Input(shape=(row, col, ch), name='img_in')
    x = img_in
    #x = Cropping2D(cropping=((10,0), (0,0)))(x) #trim 10 pixels off top
    #x = Lambda(lambda x: x/127.5 - 1.)(x) # normalize and re-center
    x = Lambda(lambda x: x/255.0)(x)
    x = Conv2D(24, (5,5), strides=(2,2), activation='relu', name="conv2d_1")(x)
    x = Dropout(drop)(x)
    x = Conv2D(32, (5,5), strides=(2,2), activation='relu', name="conv2d_2")(x)
    x = Dropout(drop)(x)
    x = Conv2D(64, (5,5), strides=(2,2), activation='relu', name="conv2d_3")(x)
    x = Dropout(drop)(x)
    x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name="conv2d_4")(x)
    x = Dropout(drop)(x)
    x = Conv2D(64, (3,3), strides=(1,1), activation='relu', name="conv2d_5")(x)
    x = Dropout(drop)(x)
    
    x = Flatten(name='flattened')(x)
    x = Dense(100, activation='relu')(x)
    #x = Dropout(drop)(x)
    x = Dense(50, activation='relu')(x)
    #x = Dropout(drop)(x)

    outputs = []
    outputs.append(Dense(num_outputs, activation='linear', name='steering_throttle')(x))
    
        
    model = Model(inputs=[img_in], outputs=outputs)
    opt = Adam(lr=0.0001)
    model.compile(optimizer=opt, loss="mse", metrics=['acc'])
    return model

go("nvidia1", epochs=10, inputs='./dataset/testing/*.jpg')

#18. Load trained model ai-av-model.h5

In [ ]:
# !ls
from keras.models import load_model
model=load_model('ai-av-model.h5')
model.compile("sgd", "mse")	

#19. Predict (no pre-processing)
In this case, since model ai-av-model.h5 was trained with no pre-processing, the expectation is this case will produce the more accurate results.



In [ ]:
filemask = 'dataset/testing/*.jpg'
files = GetJPGFiles(filemask)
predNoPreProc = []
for file in files:
  fullpath = file
  img_arr = cv2.imread(fullpath)
  img_arr = cv2.resize(img_arr, (160, 120), cv2.INTER_AREA)
  img_arr = img_arr.reshape((1,) + img_arr.shape)
  mypred = model.predict(img_arr)
  predNoPreProc.append(mypred[0][0]) # append steering angle only
print('*** Predictions completed ***')  

#20. Plot results

In [ ]:
# plot
pnp = np.asarray(predNoPreProc)
# print(type(g)) # <class 'numpy.ndarray'>
plt.rcParams["figure.figsize"] = (18,3)
nc = 25 # norm. constant, maximum steering angle

plt.plot(pnp*nc)
# plt.plot(sarr[:,1]*25, label="simulator")

plt.ylabel('Steering angle')
plt.xlabel('Frame number')    
# Set a title of the current axes.
mytitle = 'Predicted steering angles by ai-av-model.h5 for logs_Wed_Nov_25_23_39_22_2020'
plt.title(mytitle)
plt.grid(axis='y')
# set limit
plt.xlim([-5,len(pnp)+5])
plt.gca().invert_yaxis()
plt.show()

#21. Predict (with pre-processing)

In [ ]:
files = GetJPGFiles(filemask)
predPreProc = []
for file in files:
  fullpath = file
  img_arr = preprocess(fullpath)
  img_arr = img_arr.reshape((1,) + img_arr.shape)
  mypred = model.predict(img_arr)
  predPreProc.append(mypred[0][0]) # append steering angle only
print('*** Predictions with pre-processing completed ***')

#22. Plot results

In [ ]:
# plot
ppr = np.asarray(predPreProc)
# print(type(g)) # <class 'numpy.ndarray'>
plt.rcParams["figure.figsize"] = (18,3)
nc = 25 # norm. constant, maximum steering angle

plt.plot(ppr*nc)
# plt.plot(sarr[:,1]*25, label="simulator")

plt.ylabel('Steering angle')
plt.xlabel('Frame number')    
# Set a title of the current axes.
mytitle = 'Predicted steering angles by ai-av-model.h5 for logs_Wed_Nov_25_23_39_22_2020, with image pre-processing'
plt.title(mytitle)
plt.grid(axis='y')
# set limit
plt.xlim([-5,len(ppr)+5])
plt.gca().invert_yaxis()
plt.show()

#23. Goodness-of-steer for our Colab trained model

In [ ]:
fpnp = gos(pnp, g, 25)
fppr = gos(ppr, g, 25)
str_pnp = "{:.2f}".format(fpnp)
str_ppr = "{:.2f}".format(fppr)
print("gs, no pre-processing = ", str_pnp)
print("gs, with pre-processing = ", str_ppr)

#24. Overlay plots for further analysis

In [ ]:
plotMultipleSteeringAngles(pnp, ppr, g, 25, save=False, track= "Generated Track", mname="ai-av-model.h5", w=18, h=3)


#Recap
* We used a pre-trained Tensorflow/Keras model and also trained a model to predict steering angles given a set of images (taken from a lap around a track generated by the Unity game engine, using the SDSandbox environment).
* This is known as a "regression" task, where a continuous value (steering angle) is prediction, as opposed to "classification" tasks, we have tackled in previous workshops, where an image was classified as [fire/no-fire](https://github.com/CityDataScienceSociety/ComputerVisionWorkshops/tree/main/detect-fire-with-AI), or and audio file was classified as the words [yes or no](https://github.com/CityDataScienceSociety/NLP-Workshop).
* The same image size used for training must be used at "inference" time, for predicting.
* The model could be deployed onto the [DIY Robocars](https://diyrobocars.com/) model scale platform, if trained with the appropriate track dataset (e.g. Warehouse).
# Thanks for taking part! :)

